In [1]:
# Shhhhhhh......
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# Other imports
import pandas as pd

In [2]:
# Get data from mongoDB
import pymongo
import ezmongo
df = ezmongo.get_data("MSFT","cleaned_trained_scaled")

In [3]:
#I solating the columns we want.
# Lets try all data points
input_df = df.drop(['Date','BuyOrSell'], axis=1)

output_df = df['BuyOrSell'].values

In [4]:
from keras.utils import to_categorical

# Step 2: One-hot encoding
# Even though the last columns is already 1 or 0, to_categorical needs to somehow convert this into 
# an array that looks like [x1,x2] and literally separate them into different states
# Try printing out output and outputs_df to see the difference.
output_hot_shot_encoded = to_categorical(output_df)

#Creating the NN. Import the stuff

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 

Using TensorFlow backend.


In [7]:
#EDIT THIS SECTION OF CODE TO DESIGN YOUR OWN NEURAL NET.

#Create the model
model = Sequential()

#Setting general NN info here

number_of_inputs = len(input_df.columns)

number_of_layers = number_of_inputs + 10

number_of_neurons_per_layer = number_of_layers + 10


#Adding the first input layer
model.add(Dense(units=number_of_neurons_per_layer, activation='relu', input_dim=number_of_inputs))

for i in range(number_of_layers):
    model.add(Dense(units=number_of_neurons_per_layer, activation='relu'))

#Adding the output layer
model.add(Dense(units=2, activation='softmax')) 
#Compiling
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

In [ ]:
#Training the neurons

model.fit(
    input_df,
    output_hot_shot_encoded,
    epochs=1000,
    shuffle=True,
    verbose=2
)

Epoch 1/1000
8398/8398 - 2s - loss: 0.6910 - acc: 0.5376
Epoch 2/1000
8398/8398 - 1s - loss: 0.6908 - acc: 0.5381
Epoch 3/1000
8398/8398 - 2s - loss: 0.6899 - acc: 0.5381
Epoch 4/1000
8398/8398 - 1s - loss: 0.6902 - acc: 0.5381
Epoch 5/1000
8398/8398 - 2s - loss: 0.6906 - acc: 0.5381
Epoch 6/1000
8398/8398 - 2s - loss: 0.6905 - acc: 0.5381
Epoch 7/1000
8398/8398 - 2s - loss: 0.6906 - acc: 0.5381
Epoch 8/1000
8398/8398 - 2s - loss: 0.6905 - acc: 0.5381
Epoch 9/1000
8398/8398 - 2s - loss: 0.6904 - acc: 0.5381
Epoch 10/1000
8398/8398 - 2s - loss: 0.6903 - acc: 0.5381
Epoch 11/1000
8398/8398 - 1s - loss: 0.6905 - acc: 0.5381
Epoch 12/1000
8398/8398 - 1s - loss: 0.6904 - acc: 0.5381
Epoch 13/1000
8398/8398 - 1s - loss: 0.6904 - acc: 0.5381
Epoch 14/1000
8398/8398 - 1s - loss: 0.6904 - acc: 0.5381
Epoch 15/1000
8398/8398 - 1s - loss: 0.6904 - acc: 0.5381
Epoch 16/1000
8398/8398 - 2s - loss: 0.6905 - acc: 0.5381
Epoch 17/1000
8398/8398 - 1s - loss: 0.6904 - acc: 0.5381
Epoch 18/1000
8398/8398

In [ ]:
# model.predict is what we use to see the neural net's output on real data
results = model.predict(input_df, verbose=2)

# This part basically will create an array with a bunch of [X,Y] inside.
# Each [X,Y] pair represents the probability of the AI deciding to buy or sell on that day. 

In [ ]:
# Just to prove that the length of the results is equal to the length of the inputs. 
len(results) == len(input_df)

In [ ]:
#SANDBOXXXXXXXX
df2 = df['BuyOrSell'].to_frame()

# What the heck is the order of the results?
# Lets split it up and try to find out how much sense it makes
col1, col2 = zip(*results)


# READ THIS PART LATER: 
# So the original sign was ">" because it was assumed that the first column was the probability
# to buy. So if X (the first col) was > than Y, the AI basically bought.
# This is not true however, because it's backwards! The second column is whether or not it buys and the first
# is sell. I believe this has to do with the way hot-encoding works and how a 
# 0 becomes a 1,0 while a 1 becomes a 0,1.
# The sign has been switched to "<" and now it's finally correct
def mapingFunc(x,y):
    if x < y:
        return 1
    else:
        return 0

# If the first col represents buy then when the first col's number is greater
# than the second col's number, 
result = list(map(mapingFunc, col1, col2)) 

In [ ]:
#SANDBOX STUFF. Messing around to try to find out exactly which number in the keras.predict means what

df2['AIchoice'] = result
df2.head(1)

right_choice = 0
wrong_choice = 0
for index, row in df2.iterrows():
    if row['BuyOrSell'] == row['AIchoice']:
        right_choice += 1
    else:
        wrong_choice += 1

In [ ]:
# So the total number of choices matches the inputDF. 
# This is good. No rows are missing
right_choice + wrong_choice

In [ ]:
# So what's the accuracy? It should be around 60% based on the verbose keras.fit
right_choice / (right_choice + wrong_choice)
# READ LATER: It was inverted and showing 40% accuracy. But since switching the signs, it is now correct.

In [ ]:
post_train_df = df2

import ezmongo
raw = ezmongo.get_data('GOOG', 'cleaned')

In [ ]:
raw['BuyOrSell'] = df2['BuyOrSell']
raw['AIchoice'] = df2['AIchoice']


upload = {"data":raw.to_dict('records')}
upload['id'] = 0
upload['description'] = "A standard feed forward NN that looked at today's high, low, open, closing, adjusted closing, and volumne and tried to use that data to predict tomorrow's price. It was trained on GOOG from 2004 to 2019 and even when ran on the same data, it only had 60% accuracy."

In [ ]:
ezmongo.set_data(raw, 'GOOG','AI')